In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import stim
from stimrgs_v1.utils import *
from stimrgs_v1.stabilizer_operations import *
from tqdm import tqdm

In [ ]:
def find_stabilizers_result(num_nodes:int):
    all_stabilizer_result = []
    num_nodes_each_row = int(num_nodes/4)
    row_1 = list(range(0, num_nodes_each_row))
    row_2 = list(range(num_nodes_each_row, 2*num_nodes_each_row))
    row_3 = list(range(2*num_nodes_each_row, 3*num_nodes_each_row))
    row_4 = list(range(3*num_nodes_each_row, 4*num_nodes_each_row))

    for i in row_1:
        for j in row_2:
            for k in row_3:
                for l in row_4:
                    # Create s_1 and replace character at index j with 'Z'
                    s_1 = '_' * i + 'X' + '_' * (11 - i) 
                    s_1 = s_1[:j] + 'Z' + s_1[j+1:]  # Create a new string with 'Z' at the desired position                

                    # Create s_2 and replace character at index k with 'Z'
                    s_2 = '_' * j + 'X' + '_' * (11 - j)
                    s_2 = s_2[:k] + 'Z' + s_2[k+1:]  # Create a new string with 'Z' at the desired position
                    # Create a new string with 'Z' at the desired index i
                    s_2 = s_2[:i] + 'Z' + s_2[i+1:]  # Replace character at index i with 'X'

                    s_3 = '_' * k + 'X' + '_' * (11 - k)
                    s_3 = s_3[:l] + 'Z' + s_3[l+1:]  
                    # Create a new string with 'Z' at the desired position
                    s_3 = s_3[:j] + 'Z' + s_3[j+1:]  # Replace character at index i with 'X'

                    s_4 = '_' * l + 'X' + '_' * (11 - l)
                    s_4 = s_4[:k] + 'Z' + s_4[k+1:]  # Create a new string with 'Z' at the desired position

                    stabilizer_result = [s_1, s_2, s_3, s_4]
                    all_stabilizer_result.append(stabilizer_result)

    if len(all_stabilizer_result) != ((num_nodes//4)**4):
        raise ValueError("The number of stabilizer strings is not correct")
    
    return all_stabilizer_result

In [25]:
# Configuration parameters
num_trial = 10
num_nodes = 12
num_bell_between_row = 2

max_found_bells = 0
stabilizers_results = []
stabilizers_results = find_stabilizers_result(num_nodes=num_nodes)

for i in tqdm(range(num_trial)):

    for num_measurements in range(1, num_nodes-(num_nodes//4)): 
        # print(f'Measure: {num_measurements} nodes')
        combinations = generate_combinations_with_adjustable_replacement(
            nodes=[i for i in range(num_nodes)], 
            max_length=num_measurements, # You have to change this parameter, if you want to modify number of node operation
            max_self_combination=1
        )    

        for c in combinations:

            # Restart parameters before operate new measurement set
            found: bool = False
            bell_found_list = []

            q, circuit = generate_rgs_random(num_nodes=num_nodes, num_bell_between_row=num_bell_between_row)
            s = stim.TableauSimulator()
            s.do_circuit(stim.Circuit(circuit))

            # Operate measurement
            for index in c:
                # Update s, q
                s, q = measure_z_with_correction(index=index, s=s, queries=q)

            # Check if result match one of all possible results
            for result in stabilizers_results:
                if is_subset(result, q):
                    # print(f"Found solution: {result}")   
                    bell_found_list.append(result)                     
                    found = True                       
            
            if len(bell_found_list) >= max_found_bells:
                max_found_bells = len(bell_found_list)

    print(f'Max found bells {i}: {max_found_bells}')

 10%|█         | 1/10 [00:05<00:48,  5.38s/it]

Max found bells 0: 2


 20%|██        | 2/10 [00:10<00:43,  5.39s/it]

Max found bells 1: 2


 30%|███       | 3/10 [00:15<00:36,  5.26s/it]

Max found bells 2: 2


 30%|███       | 3/10 [00:17<00:41,  6.00s/it]


KeyboardInterrupt: 

In [27]:
experiments_num_nodes = [12, 16, 20, 24, 28, 32, 36, 40, 44, 48]
experiments_num_bell_between_row = [1, 2, 3, 4]
num_trial = 5

for num_nodes in tqdm(experiments_num_nodes):
    for num_bell_between_row in experiments_num_bell_between_row:
        
        max_found_bells = 0
        stabilizers_results = []
        stabilizers_results = find_stabilizers_result(num_nodes=num_nodes)

        for i in range(num_trial):

            for num_measurements in range(1, num_nodes-(num_nodes//4)): 
                # print(f'Measure: {num_measurements} nodes')
                combinations = generate_combinations_with_adjustable_replacement(
                    nodes=[i for i in range(num_nodes)], 
                    max_length=num_measurements, # You have to change this parameter, if you want to modify number of node operation
                    max_self_combination=1
                )    

                for c in combinations:

                    # Restart parameters before operate new measurement set
                    found: bool = False
                    bell_found_list = []

                    q, circuit = generate_rgs_random(num_nodes=num_nodes, num_bell_between_row=num_bell_between_row)
                    s = stim.TableauSimulator()
                    s.do_circuit(stim.Circuit(circuit))

                    # Operate measurement
                    for index in c:
                        # Update s, q
                        s, q = measure_z_with_correction(index=index, s=s, queries=q)

                    # Check if result match one of all possible results
                    for result in stabilizers_results:
                        if is_subset(result, q):
                            # print(f"Found solution: {result}")   
                            bell_found_list.append(result)                     
                            found = True                       
                    
                    if len(bell_found_list) >= max_found_bells:
                        max_found_bells = len(bell_found_list) 

        print(f'num_nodes: {num_nodes}')
        print(f'num_bell_between_row: {num_bell_between_row}')
        print(f'Max found bells: {max_found_bells}')

  0%|          | 0/10 [00:00<?, ?it/s]

num_nodes: 12
num_bell_between_row: 1
Max found bells: 2
num_nodes: 12
num_bell_between_row: 2
Max found bells: 2


  0%|          | 0/10 [01:17<?, ?it/s]

num_nodes: 12
num_bell_between_row: 3
Max found bells: 1


ValueError: Number of Bell pairs should be less than or equal to the number of nodes in each row.

In [39]:
# Configuration parameters
num_trial = 10
num_nodes = 16
num_bell = 2

stabilizers_results = []
stabilizers_results = find_stabilizers_result(num_nodes=num_nodes)

# Reset max_found_bells every time we change num_bell_between_row
max_found_bells = 0

for _ in tqdm(range(num_trial)):        

    # Constant number of measurements
    num_measurements = 4
    
    # print(f'Measure: {num_measurements} nodes')
    combinations = generate_combinations_with_adjustable_replacement(
        nodes=[i for i in range(num_nodes)], 
        max_length=num_measurements, # You have to change this parameter, if you want to modify number of node operation
        max_self_combination=1
    )    

    for c in combinations:

        # Restart parameters before operate new measurement set
        found: bool = False
        bell_found_list = []

        # q, circuit = generate_rgs_random(num_nodes=num_nodes, num_bell_between_row=num_bell)
        # s = stim.TableauSimulator()
        # s.do_circuit(stim.Circuit(circuit))
        s = stim_rgs_4arms_2bells()
        q = [
            "X___ZZ__________",  # node 0
            "_X__Z_Z_________",  # node 1
            "__X__Z_Z________",  # node 2
            "___X__ZZ________",  # node 3
            "ZZ__X___Z_______",  # node 4
            "Z_Z__X___Z______",  # node 5
            "_Z_Z__X___Z_____",  # node 6
            "__ZZ___X___Z____",  # node 7
            "____Z___X___ZZ__",  # node 8
            "_____Z___X__Z_Z_",  # node 9
            "______Z___X__Z_Z",  # node 10
            "_______Z___X__ZZ",  # node 11
            "________ZZ__X___",  # node 12
            "________Z_Z__X__",  # node 13
            "_________Z_Z__X_",  # node 14
            "__________ZZ___X",  # node 15
        ]

        # Operate measurement
        for index in c:
            # Update s, q
            s, q = measure_z_with_correction(index=index, s=s, queries=q)

        # Check if result match one of all possible results
        for result in stabilizers_results:
            if is_subset(result, q):
                # print(f"Found solution: {result}")   
                bell_found_list.append(result)                     
                found = True                       
            
        # Update when found new high
        if len(bell_found_list) >= max_found_bells:
            max_found_bells = len(bell_found_list)

    print(f'max_found_bells trial: {max_found_bells}')

print(f'Max found bells {num_bell}: {max_found_bells}')

 10%|█         | 1/10 [00:01<00:09,  1.11s/it]

max_found_bells trial: 0


 20%|██        | 2/10 [00:02<00:09,  1.15s/it]

max_found_bells trial: 0


 30%|███       | 3/10 [00:03<00:07,  1.12s/it]

...

max_found_bells trial: 0


 90%|█████████ | 9/10 [00:10<00:01,  1.10s/it]

max_found_bells trial: 0


100%|██████████| 10/10 [00:11<00:00,  1.14s/it]

max_found_bells trial: 0
Max found bells 2: 0


In [3]:
# Configuration parameters
num_trial = 5
num_nodes = 16
num_bell_between_row = [i+2 for i in range(num_nodes//4 -1)]

stabilizers_results = []
stabilizers_results = find_stabilizers_result(num_nodes=num_nodes)

for num_bell in num_bell_between_row:

    # Reset max_found_bells every time we change num_bell_between_row
    max_found_bells = 0

    for _ in range(num_trial):

        stabilizers_results = []
        stabilizers_results = find_stabilizers_result(num_nodes=num_nodes)

        for num_measurements in range(1, num_nodes-(num_nodes//4)-4):
            # print(f'Measure: {num_measurements} nodes')
            combinations = generate_combinations_with_adjustable_replacement(
                nodes=[i for i in range(num_nodes)], 
                max_length=num_measurements, # You have to change this parameter, if you want to modify number of node operation
                max_self_combination=1
            )    

            for c in combinations:

                # Restart parameters before operate new measurement set
                found: bool = False
                bell_found_list = []

                q, circuit = generate_rgs_random(num_nodes=num_nodes, num_bell_between_row=num_bell)
                s = stim.TableauSimulator()
                s.do_circuit(stim.Circuit(circuit))

                # Operate measurement
                for index in c:
                    # Update s, q
                    s, q = measure_z_with_correction(index=index, s=s, queries=q)

                # Check if result match one of all possible results
                for result in stabilizers_results:
                    if is_subset(result, q):
                        # print(f"Found solution: {result}")   
                        bell_found_list.append(result)                     
                        found = True                       
                
                # Update when found new high
                if len(bell_found_list) >= max_found_bells:
                    max_found_bells = len(bell_found_list)

    print(f'Max found bells {num_bell}: {max_found_bells}')

Max found bells 2: 3
Max found bells 3: 1
Max found bells 4: 0
